In [16]:
%load_ext autoreload
%autoreload 2
import manenv
from manenv.utils import *
from manenv.world import World
from manenv.component import *
from manenv.render_wrapper import RenderWrapper
from manenv.product import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
world = World((3, 3))
s = np.zeros((5, 5), dtype=np.int8)
s[1][2] = 1
s[2][3] = 2
product = Product(s)
world.place_component(make_vector(1, 1), Spawner(product))
world.update()

wrapper = RenderWrapper(world)
wrapper.render()